In [1]:
from decouple import config

In [2]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [3]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [4]:
import requests

params = {
    "api_key" : ALPHA_VANTAGE_API_KEY,
    "ticker" : "AAPL",
    "function" : "TIME_SERIES_INTRADAY"
}

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=5min&apikey={api_key}'.format(**params)
r = requests.get(url)
data = r.json()

#data

In [8]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (5min)'

In [29]:
results = data[dataset_key]  # dictionary

In [24]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-12-03 19:55:00'

In [25]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '242.9600',
 '2. high': '243.0000',
 '3. low': '242.8700',
 '4. close': '242.9350',
 '5. volume': '8813'}

In [28]:
from datetime import datetime
import pytz
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result_val):
    #unix_timestamp = result.get('t')/1000
    #utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone("UTC"))
    #print(utc_timestamp)
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    # timestamp
    return{
        "open_price" : Decimal(result_val["1. open"]),
        "close_price" : Decimal(result_val["4. close"]),
        "high_price" : Decimal(result_val["2. high"]),
        "low_price" : Decimal(result_val["3. low"]),
        "number_of_trades" : None,
        "volume" : int(result_val["5. volume"]),
        "volume_weighted_average" : None,
        "time" : timestamp,
    }

transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('242.9600'),
 'close_price': Decimal('242.9350'),
 'high_price': Decimal('243.0000'),
 'low_price': Decimal('242.8700'),
 'number_of_trades': None,
 'volume': 8813,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 12, 4, 0, 55, tzinfo=<UTC>)}

In [31]:
for times_stamp in results.keys():
    x = transform_alpha_vantage_result(times_stamp, results.get(times_stamp))
    print(x)

{'open_price': Decimal('242.9600'), 'close_price': Decimal('242.9350'), 'high_price': Decimal('243.0000'), 'low_price': Decimal('242.8700'), 'number_of_trades': None, 'volume': 8813, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 4, 0, 55, tzinfo=<UTC>)}
{'open_price': Decimal('242.9681'), 'close_price': Decimal('242.9600'), 'high_price': Decimal('242.9700'), 'low_price': Decimal('242.9300'), 'number_of_trades': None, 'volume': 1523, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 4, 0, 50, tzinfo=<UTC>)}
{'open_price': Decimal('242.9000'), 'close_price': Decimal('242.9313'), 'high_price': Decimal('242.9700'), 'low_price': Decimal('242.8400'), 'number_of_trades': None, 'volume': 4167, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 4, 0, 45, tzinfo=<UTC>)}
{'open_price': Decimal('242.8700'), 'close_price': Decimal('242.8950'), 'high_price': Decimal('242.9500'), 'low_price': Decimal('242.8200'), 'number_of_trades': None, 'v